# Import the required Python Modules (and modules created for BinMSGUI)

In [10]:
import os
import numpy as np
import sys
import struct
import json
import datetime
import scipy
import scipy.signal
import matplotlib.pyplot as plt
%matplotlib inline

# this will obtain the path for the notebook
# if you have moved the notebook, you will need to set the notebook_path to the
# .../BinMSGUI/BinMSGUI/jupyter directory
notebook_path = os.path.dirname(os.path.abspath("__file__"))

# the code path is two folders up from this notebook + /code
core_path = os.path.dirname(notebook_path)
basepath = os.path.dirname(os.path.dirname(notebook_path))

# add the core_path and basepath so we can import core.module_name
sys.path.append(core_path)
sys.path.append(basepath)

# import the binMSGUI modules
from core.readMDA import readMDA, get_Fs
from core.readBin import get_bin_data, get_raw_pos, get_active_tetrode, get_channel_from_tetrode, get_active_eeg
from core.Tint_Matlab import int16toint8, get_setfile_parameter
from core.tetrode_conversion import convert_tetrode, batch_basename_tetrodes
from core.convert_position import convert_position
from core.eeg_conversion import convert_eeg
from core.utils import find_sub, find_bin_basenames
from core.bin2mda import convert_bin2mda
from core.mdaSort import sort_bin, run_sort
from core.set_conversion import convert_setfile
from core.intan_mountainsort import validate_session, convert_bin_mountainsort

# Choose Directory Containing .Bin Files to Analyze

In [4]:
# directory that you want to analyze
directory = r'/mnt/d/freelance-work/catalyst-neuro/hussaini-lab-to-nwb/example_data_raw/data_5s'

# finds the basenames within this directory
basenames = find_bin_basenames(directory)

# prints all the basenames
print('Found %d basename(s) within this directory: %s' % (len(basenames), directory))
print('------------------------')
for name in basenames:
    print(name)

Found 1 basename(s) within this directory: /mnt/d/freelance-work/catalyst-neuro/hussaini-lab-to-nwb/example_data_raw/data_5s
------------------------
axona_raw_5s


## Analysis Parameters
This cell will contain all the parameters that are required for the analysis. There are quite a few so make sure that you check that the parameters are correct before proceeding.

In [5]:
########### filtering parameters ###########
# The .bin data is already filtered based upon whatever settings you chose in the dacqUSB, therefore this
# is just for show and does not do anything. If you want me to add an actual filtering step feel free to
# let me know, right now this is just for show.

freq_min = 300  # -units Hz- default 300
freq_max = 7000  # -units Hz- default 7000
notch_filter = False  # the data is already notch filtered likely

########## Clip Parameters ##############
pre_spike = 15
post_spike = 35
clip_size = 50 # this needs to be left at 50 for Tint, Tint only likes 50 samples
if (pre_spike + post_spike != clip_size) or (clip_size != 50):
    raise Exception('The pre_spike and the post_spike must add up to be 50, and the clip size must remain at 50 for Tint!')

############ whiten ####################
# do you want to whiten the data? The MountainSort Paper suggests that spatial whitening is crucial
# for separating nearby clusters since it will remove any correlations among channels. Keep in mind
# that spatial whitening will also normalize your data so when you threshold you will be thresholding
# based off of # of standard deviations, and not a bit/uV value.
# if you want to whiten, set whiten='true', if you do not set whiten='false'
whiten = 'true' 
# whiten = 'false'

############# detect_sign ################
# recommend only doing positive peaks so we don't get any weird issues with a cell that is
# aligned with the peak, and seemingly the same cell aligned with the trough (in this case
# both peak and trough would have to exceed the threshold).

# detect_sign = 0  # positive or negative peaks
detect_sign = 1  # only positive peaks
# detect_sign = -1  # only negative peaks

########## detect_interval ##############
# the algorithm will take the detect_interval value and bin the data in bin sizes of that many
# samples. Then it will find the peak (or trough, or both) of each bin and evaluate that event
# if it exceeds the threshold value. Therefore the detect_interval is roughly the number of 
# samples between the events (peaks/troughs depending on your detect_sign)

# default detect_interval is 50

# detect_interval = 50 
detect_interval = 20  

############ detect_threshold ###############

# threshold values, I changed it into a whitened and non whitened threshold
# this is because if you whiten the data you normalize it by the variance, thus
# a threshold of 3 is essentially saying 3 standard deviations. However if you do not whiten
# the data is not normalized and thus, you would be using a bit value, maybe should take whatever
# value is in the threshold from the set file.

automate_threshold = False  # Don't Change this, here we are just initializing the automate_threshold value to False by default

if whiten == 'true':
    # detect_threshold = 3  # units: ~sd's
    detect_threshold = 4  # units: ~sd's
    
else:
    # this mean's the data was not whitened
    
    detect_threshold = 13000  #  units: bits 
    
    # if you want to find the threshold from the .set file and use that 
    # set automate_threshold to True, otherwise False. This threshold would override any
    # value set above. I'd recommend setting this to true as this is variable from .set file
    # to .set file it seems.
    # automate_threshold = True 
    automate_threshold = False

# ########### artifact masking parameters ###########
# here we bin the data into masked_chunk_size bins, and it will take the sqrt of the sum of 
# the squares (RSS) for each bin. It will then find the SD for all the bins, and if the bin is
# above mask_threshold SD's from the average bin RSS, it will consider it as high amplitude noise
# and remove this chunk (and neighboring chunks).

# mask = True  # set the value to True if you want to include the artifact masking step
mask = False  # set the value to False if you want to skip the artifact masking step
mask_threshold = 6  #  units: SD's, the threshold that once exceed the bin/chunk will be zero'ed (and the neighbors)

# the size of the bins/chunks to use when binning the data. If the value is set to  None 
# the defaul tmasked_chunk_size it will default to Fs/20
masked_chunk_size = None  

mask_num_write_chunks = 100  # how many chunks will be simultaneously written to the masked output file

########## Feature/PCA Parameters ##########
num_features = 10
max_num_clips_for_pca = 1000

# random parameters, probably don't need to change
self = None  # don't worry about this, this is for objective oriented programming (my GUIs)

## Runs Analysis on each Basename

In [11]:
directory

'/mnt/d/freelance-work/catalyst-neuro/hussaini-lab-to-nwb/example_data_raw/data_5s'

In [13]:
os.path.basename(current_basename)

'axona_raw_5s'

In [6]:
for i, current_basename in enumerate(basenames):
    print('Analyzing set_file %d/%d: ' % (i+1, len(basenames)))
    
    if whiten != 'true' and automate_threshold:
        # then you decided you want to automatically get the threshold from the .set file
        set_filename = '%s.set' % os.path.join(directory, current_basename)
        detect_threshold = int(get_setfile_parameter('threshold', set_filename))
    
    print('Using the following detect_threshold: %.2f' % (float(detect_threshold)))
        
    convert_bin_mountainsort(directory, current_basename, whiten=whiten, 
                             detect_interval=detect_interval,
                             detect_sign=detect_sign, 
                             detect_threshold=detect_threshold, 
                             freq_min=freq_min,
                             freq_max=freq_max, mask_threshold=mask_threshold, 
                             masked_chunk_size=masked_chunk_size,
                             mask_num_write_chunks=mask_num_write_chunks, 
                             clip_size=clip_size, 
                             mask=mask,
                             num_features=num_features,
                             max_num_clips_for_pca=max_num_clips_for_pca,
                             pre_spike=pre_spike, post_spike=post_spike,
                             notch_filter=notch_filter, self=self)
    
    print('-------------------')
    
print('Finished Analysis')

Analyzing set_file 1/1: 
Using the following detect_threshold: 4.00
[2021-04-27 10:07:42]: Converting the following tetrode: 1!
[2021-04-27 10:07:47]: Converting the following tetrode: 2!
[2021-04-27 10:07:55]: Converting the following tetrode: 3!
[2021-04-27 10:08:01]: Converting the following tetrode: 4!
[2021-04-27 10:08:08]: Sorting the following file: /mnt/d/freelance-work/catalyst-neuro/hussaini-lab-to-nwb/example_data_raw/data_5s/axona_raw_5s_T1_filt.mda!
ml-run-process ms4_geoff.sort --inputs filt_fname:/mnt/mnt/d/freelance-work/catalyst-neuro/hussaini-lab-to-nwb/example_data_raw/data_5s/axona_raw_5s_t1_filt.md/mnt/d/freelance-work/catalyst-neuro/hussaini-lab-to-nwb/example_data_raw/data_5s/axona_raw_5s_T1_filt.mda --outputs firings_out:/mnt/mnt/d/freelance-work/catalyst-neuro/hussaini-lab-to-nwb/example_data_raw/data_5s/axona_raw_5s_t1_firings.md/mnt/d/freelance-work/catalyst-neuro/hussaini-lab-to-nwb/example_data_raw/data_5s/axona_raw_5s_T1_firings.mda pre_out_fname:/mnt/mnt/

FileNotFoundError: [Errno 2] No such file or directory: 'MNT:\\d\\freelance-work\\catalyst-neuro\\hussaini-lab-to-nwb\\example_data_raw\\data_5s\\axona_raw_5s_t1_terminal.tx\\mnt\\d\\freelance-work\\catalyst-neuro\\hussaini-lab-to-nwb\\example_data_raw\\data_5s\\axona_raw_5s_T1_terminal.txt'

In [24]:
from BinMSGUI.core.mdaSort import (
    get_ubuntu_path, get_windows_filename, sort_finished
)
import time

Errors seem to be related to the get_windows_filename and get_ubuntu_path functions. I don't quite see why I need them anyway, let's try without. 

In [ ]:
tint_fullpath = os.path.join(directory, current_basename)
whiten='true'
detect_interval=10
detect_sign=0
detect_threshold=3
freq_min=300
freq_max=6000
mask_threshold=6
masked_chunk_size=None
mask_num_write_chunks=100,
clip_size=50
mask=True
num_features=10
max_num_clips_for_pca=1000
self=None
verbose=True

if mask:
    mask = 'true'
else:
    mask = 'false'

tint_basename = os.path.basename(tint_fullpath)

set_filename = '%s.set' % tint_fullpath

filt_fnames = [os.path.join(directory, file) for file in os.listdir(
    directory) if '_filt.mda' in file if tint_basename in file]

for file in filt_fnames:

    msg = '[%s %s]: Sorting the following file: %s!' % \
          (str(datetime.datetime.now().date()),
           str(datetime.datetime.now().time())[:8], file)

    if self:
        self.LogAppend.myGUI_signal_str.emit(msg)
    else:
        print(msg)

    mda_basename = os.path.splitext(file)[0]
    mda_basename = mda_basename[:find_sub(mda_basename, '_')[-1]]

    firings_out = mda_basename + '_firings.mda'

    if whiten == 'true':
        pre_out_fname = mda_basename + '_pre.mda'
    else:
        pre_out_fname = None

    if mask == 'true':
        masked_out_fname = mda_basename + '_masked.mda'
    else:
        masked_out_fname = None

    metrics_out_fname = mda_basename + '_metrics.json'

    # check if these outputs have already been created, skip if they have
    existing_files = 0
    output_files = [masked_out_fname, firings_out, pre_out_fname, metrics_out_fname]
    for outfile in output_files:
        if outfile is not None:
            if os.path.exists(outfile):
                existing_files += 1

    if existing_files == len(output_files):
        msg = '[%s %s]: The following file has already been sorted: %s, skipping sort!#Red' % \
              (str(datetime.datetime.now().date()),
               str(datetime.datetime.now().time())[:8], file)

        if self:
            self.LogAppend.myGUI_signal_str.emit(msg)
        else:
            print(msg)
        continue

    terminal_text_filename = mda_basename + '_terminal.txt'

    filt_fname = file

    Fs = int(get_setfile_parameter('rawRate', set_filename))

    if masked_chunk_size is None:
        masked_chunk_size = int(Fs/20)

    sorting = True
    sorting_attempts = 0

    if os.path.exists(terminal_text_filename):
        os.remove(terminal_text_filename)

    while sorting:

        run_sort(filt_fname=filt_fname,
                 pre_out_fname=pre_out_fname,
                 metrics_out_fname=metrics_out_fname,
                 firings_out=firings_out,
                 masked_out_fname=masked_out_fname,
                 samplerate=Fs,
                 detect_interval=detect_interval,
                 detect_sign=detect_sign,
                 detect_threshold=detect_threshold,
                 freq_min=freq_min,
                 freq_max=freq_max,
                 mask_threshold=mask_threshold,
                 mask_chunk_size=masked_chunk_size,
                 mask_num_write_chunks=mask_num_write_chunks,
                 whiten=whiten,
                 mask_artifacts=mask,
                 clip_size=clip_size,
                 num_features=num_features,
                 max_num_clips_for_pca=max_num_clips_for_pca,
                 terminal_text_filename=terminal_text_filename,
                 verbose=verbose)

        # wait for the sort to finish before continuing
        finished, sort_code = sort_finished(terminal_text_filename)

        if sorting_attempts >= 5:
            # we've tried to sort a bunch of times, doesn't seem to work
            sorting = False

        elif 'Abort' in sort_code:
            # there's a problem with the sort,
            sorting = False
            msg = '[%s %s]: There was an error sorting the following file, consult terminal text file: %s!#Red' % \
                  (str(datetime.datetime.now().date()),
                   str(datetime.datetime.now().time())[:8], filt_fname)

            if self:
                self.LogAppend.myGUI_signal_str.emit(msg)
            else:
                print(msg)

        elif not finished:
            time.sleep(0.5)
            os.remove(terminal_text_filename)
            sorting_attempts += 1

        else:
            sorting = False

[2021-04-27 11:01:17]: Sorting the following file: /mnt/d/freelance-work/catalyst-neuro/hussaini-lab-to-nwb/example_data_raw/data_5s/axona_raw_5s_T1_filt.mda!
ml-run-process ms4_geoff.sort --inputs filt_fname:/mnt/d/freelance-work/catalyst-neuro/hussaini-lab-to-nwb/example_data_raw/data_5s/axona_raw_5s_T1_filt.mda --outputs firings_out:/mnt/d/freelance-work/catalyst-neuro/hussaini-lab-to-nwb/example_data_raw/data_5s/axona_raw_5s_T1_firings.mda pre_out_fname:/mnt/d/freelance-work/catalyst-neuro/hussaini-lab-to-nwb/example_data_raw/data_5s/axona_raw_5s_T1_pre.mda metrics_out_fname:/mnt/d/freelance-work/catalyst-neuro/hussaini-lab-to-nwb/example_data_raw/data_5s/axona_raw_5s_T1_metrics.json masked_out_fname:/mnt/d/freelance-work/catalyst-neuro/hussaini-lab-to-nwb/example_data_raw/data_5s/axona_raw_5s_T1_masked.mda --parameters freq_min:300 freq_max:6000 samplerate:48000 detect_sign:0 adjacency_radius:-1 detect_threshold:3 detect_interval:10 clip_size:50 firing_rate_thresh:0.05 isolation_t

In [27]:
run_sort(filt_fname=filt_fname,
         pre_out_fname=pre_out_fname,
         metrics_out_fname=metrics_out_fname,
         firings_out=firings_out,
         masked_out_fname=masked_out_fname,
         samplerate=Fs,
         detect_interval=detect_interval,
         detect_sign=detect_sign,
         detect_threshold=detect_threshold,
         freq_min=freq_min,
         freq_max=freq_max,
         mask_threshold=mask_threshold,
         mask_chunk_size=masked_chunk_size,
         mask_num_write_chunks=mask_num_write_chunks,
         whiten=whiten,
         mask_artifacts=mask,
         clip_size=clip_size,
         num_features=num_features,
         max_num_clips_for_pca=max_num_clips_for_pca,
         terminal_text_filename=terminal_text_filename,
         verbose=verbose)

ml-run-process ms4_geoff.sort --inputs filt_fname:/mnt/mnt/d/freelance-work/catalyst-neuro/hussaini-lab-to-nwb/example_data_raw/data_5s/axona_raw_5s_t1_filt.md/mnt/d/freelance-work/catalyst-neuro/hussaini-lab-to-nwb/example_data_raw/data_5s/axona_raw_5s_T1_filt.mda --outputs firings_out:/mnt/mnt/d/freelance-work/catalyst-neuro/hussaini-lab-to-nwb/example_data_raw/data_5s/axona_raw_5s_t1_firings.md/mnt/d/freelance-work/catalyst-neuro/hussaini-lab-to-nwb/example_data_raw/data_5s/axona_raw_5s_T1_firings.mda pre_out_fname:/mnt/mnt/d/freelance-work/catalyst-neuro/hussaini-lab-to-nwb/example_data_raw/data_5s/axona_raw_5s_t1_pre.md/mnt/d/freelance-work/catalyst-neuro/hussaini-lab-to-nwb/example_data_raw/data_5s/axona_raw_5s_T1_pre.mda metrics_out_fname:/mnt/mnt/d/freelance-work/catalyst-neuro/hussaini-lab-to-nwb/example_data_raw/data_5s/axona_raw_5s_t1_metrics.jso/mnt/d/freelance-work/catalyst-neuro/hussaini-lab-to-nwb/example_data_raw/data_5s/axona_raw_5s_T1_metrics.json masked_out_fname:/mn